In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rasters as rt
from PTJPL import PTJPL, process_PTJPL_table
from PTJPL import load_Topt, load_fAPARmax

In [2]:
tower_data_filename = "combined_eco_flux_EC_filtered.csv"
tower_locations_filename = "metadata_ebc_filt.csv"

In [3]:
tower_locations_df = pd.read_csv(tower_locations_filename)
tower_locations_df

,Site ID,Name,Lat,Long,Elev,Clim,Veg,MAT,MAP,StartDate,EndDate,LE_count,closure_ratio
0,US-NC3,NC_Clearcut#3,35.7990,-76.6560,5.0,Cfa,ENF,16.6,1320.00,10/1/18 05:00,1/1/22 05:00,9576,1.02
1,PE-QFR,Quistococha Forest Reserve,-3.8344,-73.3190,104.0,Af,WET,NaN,NaN,10/1/18 05:00,1/1/20 05:00,6859,0.98
2,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,41.8222,-80.6370,270.0,Dfb,CVM,10.5,1012.70,10/1/18 05:00,12/28/19 04:00,12170,0.92
3,US-NC4,NC_AlligatorRiver,35.7879,-75.9038,1.0,Cfa,WET,16.6,1311.00,10/1/18 05:00,1/1/22 05:00,20890,0.90
4,CA-DB2,Delta Burns Bog 2,49.1190,-122.9951,4.0,NaN,WET,NaN,NaN,1/1/19 08:30,1/1/21 08:00,11884,0.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,US-xSL,"NEON North Sterling, CO (STER)",40.4619,-103.0293,1364.0,Bsk,CRO,9.7,432.67,10/1/18 07:00,12/1/22 00:00,23176,0.60
117,US-xWD,NEON Woodworth (WOOD),47.1282,-99.2414,579.0,Dfb,GRA,4.9,493.76,10/1/18 06:00,12/1/22 00:00,20494,0.60
118,US-CS4,Central Sands Irrigated Agricultural Field,44.1597,-89.5475,328.0,Dfa,CRO,7.0,830.00,1/1/20 06:30,4/27/21 06:00,6359,0.60
119,US-xAE,NEON Klemme Range Research Station (OAES),35.4106,-99.0588,516.0,Cfa,GRA,15.5,778.85,10/1/18 06:00,12/1/22 00:00,29615,0.60


In [4]:
tower_IDs = list(tower_locations_df["Site ID"])
tower_IDs

['US-NC3',
 'PE-QFR',
 'US-Mi3',
 'US-NC4',
 'CA-DB2',
 'US-Sne',
 'US-Mi1',
 'US-PFe',
 'US-NR1',
 'US-Vcp',
 'US-xAB',
 'US-HBK',
 'US-EDN',
 'US-PFh',
 'US-Me6',
 'US-NC2',
 'US-Whs',
 'US-SRM',
 'US-CS1',
 'US-PFs',
 'US-PFg',
 'US-Ha1',
 'US-Ne2',
 'US-PFm',
 'US-PFr',
 'US-PFL',
 'US-PFj',
 'US-CC2',
 'US-NR3',
 'US-PFt',
 'CA-Cbo',
 'PR-xLA',
 'US-Ne3',
 'US-PHM',
 'US-Tw1',
 'US-Hn3',
 'US-PFk',
 'US-CS2',
 'US-ONA',
 'US-CC1',
 'US-Rls',
 'US-UMd',
 'US-PFq',
 'US-PFn',
 'US-HB3',
 'US-DFC',
 'US-Wkg',
 'US-CF1',
 'US-Ho2',
 'US-Snf',
 'US-Me2',
 'US-CF3',
 'US-Ne1',
 'US-Ha2',
 'CA-Ca3',
 'US-SP1',
 'US-UMB',
 'US-Bi1',
 'US-PFi',
 'US-CF2',
 'US-KFS',
 'US-CMW',
 'US-Ro4',
 'US-PAS',
 'US-xYE',
 'US-Los',
 'US-PFd',
 'US-KM4',
 'US-Syv',
 'US-WCr',
 'US-Rws',
 'US-xJE',
 'US-SRG',
 'US-xNW',
 'US-Rms',
 'US-NR4',
 'US-xDS',
 'US-xJR',
 'US-Bar',
 'US-ALQ',
 'US-MMS',
 'US-UC1',
 'US-PFb',
 'US-xDL',
 'US-xPU',
 'US-xBL',
 'US-xST',
 'US-xSB',
 'US-xRN',
 'US-UC2',
 'US-xTR',

In [5]:
tower_names = list(tower_locations_df.Name)
tower_names

['NC_Clearcut#3',
 'Quistococha Forest Reserve',
 'LTAR UCB (Upper Chesapeake Bay) Miscanthus 3',
 'NC_AlligatorRiver',
 'Delta Burns Bog 2',
 'Sherman Island Restored Wetland',
 'LTAR UCB (Upper Chesapeake Bay) Miscanthus 1',
 'NW4 Lake-1 CHEESEHEAD 2019',
 'Niwot Ridge Forest (LTER NWT1)',
 'Valles Caldera Ponderosa Pine',
 'NEON Abby Road (ABBY)',
 'Hubbard Brook Experimental Forest',
 'Eden Landing Ecological Reserve',
 'NE2 Pine-3 CHEESEHEAD 2019',
 'Metolius Young Pine Burn',
 'NC_Loblolly Plantation',
 'Walnut Gulch Lucky Hills Shrub',
 'Santa Rita Mesquite',
 'Central Sands Irrigated Agricultural Field',
 'SE5 Aspen-5 CHEESEHEAD 2019',
 'NE1 Pine-2 CHEESEHEAD 2019',
 'Harvard Forest EMS Tower (HFR1)',
 'Mead - irrigated maize-soybean rotation site',
 'SW3 Hardwood-2 CHEESEHEAD 2019',
 'SE4 Tussock-2 CHEESEHEAD 2019',
 'SW2 Aspen-3 CHEESEHEAD 2019',
 'NE4 Maple-1 CHEESEHEAD 2019',
 'Coloma Corn 2',
 'Niwot Ridge Alpine (T-Van West)',
 'SE6 Pine-4 CHEESEHEAD 2019',
 'Ontario - Mi

In [6]:
tower_data_df = pd.read_csv(tower_data_filename)
tower_data_df

,Unnamed: 0,ID,vegetation,climate,STICinst,BESSinst,MOD16inst,PTJPLSMinst,ETinst,ETinstUncertainty,...,ESIrn_STIC,ESIrn_PTJPLSM,ESIrn_MOD16,ESIrn_BESS,ESIrn_Unc_ECO,ESIrn_LEcorr50,JET,eco_time_utc,Site Name,Date-Time
0,0,US-NC3,ENF,Cfa,270.345200,78.53355,392.851840,307.021970,487.383423,118.916280,...,0.686404,0.779526,0.997448,0.199396,0.301927,0.737734,288.683585,2019-10-02 19:09:40,US-NC3,2019-10-02 19:09:40
1,1,US-Mi3,CVM,Dfb,232.141600,229.20093,640.118470,375.089300,106.825577,167.919460,...,0.360762,0.582912,0.994784,0.356192,0.260957,0.413558,303.615450,2019-06-23 18:17:17,US-Mi3,2019-06-23 18:17:17
2,2,US-Mi3,CVM,Dfb,356.355740,335.23154,625.661700,284.686250,NaN,132.936340,...,0.566752,0.452768,0.995059,0.533156,0.211423,0.558382,345.793640,2019-06-27 16:35:42,US-Mi3,2019-06-27 16:35:42
3,3,US-Mi3,CVM,Dfb,332.938400,326.68680,624.254330,251.414490,178.827545,141.132420,...,0.531036,0.401006,0.995685,0.521065,0.225106,0.501352,329.812600,2019-06-30 15:44:10,US-Mi3,2019-06-30 15:44:10
4,4,US-Mi3,CVM,Dfb,286.854030,237.21654,511.082180,228.520170,154.791626,114.809410,...,0.559024,0.445342,0.996002,0.462290,0.223742,0.375202,262.035285,2019-07-01 14:53:48,US-Mi3,2019-07-01 14:53:48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,1060,US-xAE,GRA,Cfa,70.923310,172.37459,81.645230,15.282976,NaN,56.385185,...,0.434453,0.093619,0.500132,1.055910,0.345397,0.054837,76.284270,2021-12-11 16:01:12,US-xAE,2021-12-11 16:01:12
1061,1061,US-xAE,GRA,Cfa,116.543190,121.81641,65.469320,22.186659,NaN,40.509410,...,0.701256,0.133500,0.393938,0.732986,0.243751,0.264294,91.006255,2022-03-25 22:45:31,US-xAE,2022-03-25 22:45:31
1062,1062,US-xAE,GRA,Cfa,129.880100,0.00000,118.777240,55.343586,NaN,52.403820,...,0.550616,0.234624,0.503546,0.000000,0.222162,0.355559,87.060413,2022-04-12 22:53:09,US-xAE,2022-04-12 22:53:09
1063,1063,US-xAE,GRA,Cfa,2.707851,140.38632,126.490524,40.434025,NaN,57.769722,...,0.010924,0.163120,0.510292,0.566350,0.233056,0.269973,83.462274,2022-04-14 14:45:37,US-xAE,2022-04-14 14:45:37


In [7]:
# Create MultiPoint geometry for tower locations
tower_points = rt.MultiPoint(
    x=tower_locations_df['Long'].values,
    y=tower_locations_df['Lat'].values
)

tower_points

MULTIPOINT ((-76.656 35.799), (-73.319 -3.8344), (-80.637 41.8222), (-75.9038 35.7879), (-122.9951 49.119), (-121.7547 38.0369), (-80.6313 41.7727), (-90.3004 45.9793), (-105.5464 40.0329), (-106.5974 35.8624), (-122.3303 45.7624), (-71.7181 43.9397), (-122.114 37.6156), (-90.2406 45.9557), (-121.6078 44.3233), (-76.6685 35.803), (-110.0522 31.7438), (-110.8661 31.8214), (-89.5379 44.1031), (-90.2382 45.9381), (-90.2723 45.9735), (-72.1715 42.5378), (-96.4701 41.1649), (-90.3099 45.9207), (-90.2475 45.9245), (-90.3177 45.9409), (-90.227 45.9619), (-89.6196 44.1039), (-105.5864 40.052), (-90.2288 45.9197), (-79.9333 44.3167), (-67.0769 18.0212), (-96.4397 41.1797), (-70.8301 42.7423), (-121.6469 38.1074), (-119.4614 46.6878), (-90.3425 45.9149), (-89.5002 44.1467), (-81.9509 27.3836), (-89.6787 44.0732), (-116.7356 43.1439), (-84.6975 45.5625), (-90.2475 45.9271), (-90.2823 45.9392), (-79.2322 33.3482), (-89.7117 43.3448), (-109.9419 31.7365), (-117.0821 46.7815), (-68.747 45.2091), (-1

In [8]:
Topt = load_Topt(geometry=tower_points)
Topt

array([10.09, 25.06,  1.39,  9.74,  3.43, 11.86,  2.06,  0.  ,  0.  ,
        0.16,  6.69,  0.  , 17.53,  0.  ,  0.  , 11.61, 17.87, 18.75,
        0.  ,  0.  ,  0.  ,  0.  ,  1.85,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  , 23.22,  1.52,  3.01, 14.51, 12.82,
        0.  ,  0.  , 18.39,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , 12.81,
        0.  , 15.57,  0.  ,  0.  , 10.83,  0.  ,  0.  ,  1.85,  0.  ,
        6.34, 15.93,  0.  , 13.84,  0.  ,  0.  ,  4.28, 12.27,  0.  ,
       18.39,  0.  ,  0.  ,  0.  ,  0.64,  0.  ,  0.  ,  0.  , 13.57,
       20.45,  0.  ,  0.  ,  0.  , 17.59, 11.84,  0.  ,  0.  ,  4.4 ,
        0.72,  0.  , 11.44,  0.  ,  4.79,  0.  , 15.67,  6.  ,  0.83,
        0.  ,  0.69, 14.51, 22.28, 12.14,  0.  ,  3.79,  0.  ,  5.8 ,
        0.  ,  0.  ,  0.  , 15.81,  3.43, 12.82,  0.  ,  4.6 ,  0.  ,
        1.44,  0.  ,  2.66,  2.58, 23.21,  8.48,  4.7 , 12.81,  0.66,
        0.  ,  0.  ,  7.99,  0.  ])

In [9]:
fAPARmax = load_fAPARmax(geometry=tower_points)
fAPARmax

array([0.4659, 0.5924, 0.4865, 0.5777, 0.4949, 0.3557, 0.4792, 0.4739,
       0.4462, 0.4323, 0.604 , 0.5306, 0.2956, 0.4958, 0.4851, 0.5663,
       0.2522, 0.2773, 0.4053, 0.4951, 0.4966, 0.5545, 0.397 , 0.4877,
       0.4951, 0.4877, 0.4958, 0.4045, 0.3301, 0.4951, 0.4508, 0.5173,
       0.4027, 0.4003, 0.4334, 0.2898, 0.4877, 0.4053, 0.5321, 0.4566,
       0.2933, 0.4422, 0.4951, 0.4872, 0.5752, 0.4192, 0.2651, 0.3501,
       0.566 , 0.4022, 0.5123, 0.3514, 0.397 , 0.5545, 0.6107, 0.5694,
       0.4431, 0.4416, 0.4958, 0.3501, 0.4779, 0.2761, 0.404 , 0.5321,
       0.3157, 0.4642, 0.4739, 0.4639, 0.4913, 0.4989, 0.2906, 0.4944,
       0.3329, 0.3301, 0.3586, 0.3301, 0.5168, 0.2447, 0.5215, 0.4607,
       0.5483, 0.4962, 0.4739, 0.4725, 0.5117, 0.497 , 0.4671, 0.5047,
       0.5194, 0.5016, 0.4761, 0.4423, 0.4334, 0.409 , 0.2331, 0.4053,
       0.3176, 0.6244, 0.4664, 0.4592, 0.5215, 0.3514, 0.4322, 0.4949,
       0.2898, 0.4872, 0.4122, 0.3904, 0.4248, 0.2933, 0.5404, 0.406 ,
      

In [10]:
tower_static_data_gdf = gpd.GeoDataFrame({
    "ID": tower_IDs,
    "name": tower_names,
    "Topt": Topt,
    "fAPARmax": fAPARmax,
    "geometry": tower_points
})

tower_static_data_gdf

,ID,name,Topt,fAPARmax,geometry
0,US-NC3,NC_Clearcut#3,10.09,0.4659,POINT (-76.656 35.799)
1,PE-QFR,Quistococha Forest Reserve,25.06,0.5924,POINT (-73.319 -3.8344)
2,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865,POINT (-80.637 41.8222)
3,US-NC4,NC_AlligatorRiver,9.74,0.5777,POINT (-75.9038 35.7879)
4,CA-DB2,Delta Burns Bog 2,3.43,0.4949,POINT (-122.9951 49.119)
...,...,...,...,...,...
116,US-xSL,"NEON North Sterling, CO (STER)",0.66,0.3046,POINT (-103.0293 40.4619)
117,US-xWD,NEON Woodworth (WOOD),0.00,0.3626,POINT (-99.2414 47.1282)
118,US-CS4,Central Sands Irrigated Agricultural Field,0.00,0.3903,POINT (-89.5475 44.1597)
119,US-xAE,NEON Klemme Range Research Station (OAES),7.99,0.3387,POINT (-99.0588 35.4106)


In [11]:
for key in sorted(tower_data_df.keys()):
    print(key)

AirTempC
BESSinst
Date-Time
ESI
ESIrn_BESS
ESIrn_LEcorr50
ESIrn_MOD16
ESIrn_PTJPLSM
ESIrn_STIC
ESIrn_Unc_ECO
ETinst
ETinstUncertainty
EmisWB
G_filt
H_filt
Hcorr25
Hcorr50
Hcorr75
Hcorr_ann
ID
JET
LE
LE_filt
LEcorr25
LEcorr50
LEcorr75
LEcorr_ann
LST
LST_err
MOD16inst
NDVI
NDVI-UQ
NETRAD_filt
PET
PTJPLSMinst
RH
RH_percentage
Rg
Rn
SM
SM_rz
SM_surf
STICinst
SW_IN
Site Name
Ta
Unnamed: 0
albedo
albedo-UQ
climate
eco_time_utc
local_time
solar_hour
solar_time
time_utc
vegetation
view_zenith


In [12]:
observation_tower_IDs = list(tower_data_df.ID)
print(len(observation_tower_IDs))
observation_tower_IDs[:10]

1065


['US-NC3',
 'US-Mi3',
 'US-Mi3',
 'US-Mi3',
 'US-Mi3',
 'US-Mi3',
 'US-NC4',
 'US-Mi1',
 'US-Mi1',
 'US-Mi1']

In [13]:
model_inputs_df = pd.DataFrame({
    "ID": observation_tower_IDs,
})

model_inputs_df = pd.merge(
    left=model_inputs_df, 
    right=tower_static_data_gdf, 
    left_on="ID", 
    right_on="ID", 
    how="left"
)

model_inputs_df

,ID,name,Topt,fAPARmax,geometry
0,US-NC3,NC_Clearcut#3,10.09,0.4659,POINT (-76.656 35.799)
1,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865,POINT (-80.637 41.8222)
2,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865,POINT (-80.637 41.8222)
3,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865,POINT (-80.637 41.8222)
4,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865,POINT (-80.637 41.8222)
...,...,...,...,...,...
1060,US-xAE,NEON Klemme Range Research Station (OAES),7.99,0.3387,POINT (-99.0588 35.4106)
1061,US-xAE,NEON Klemme Range Research Station (OAES),7.99,0.3387,POINT (-99.0588 35.4106)
1062,US-xAE,NEON Klemme Range Research Station (OAES),7.99,0.3387,POINT (-99.0588 35.4106)
1063,US-xAE,NEON Klemme Range Research Station (OAES),7.99,0.3387,POINT (-99.0588 35.4106)


In [14]:
ST_K = np.array(tower_data_df.LST)
model_inputs_df["ST_K"] = ST_K
print(np.nanmin(ST_K), np.nanmean(ST_K), np.nanmax(ST_K))

258.72 302.1800469295775 359.26


In [15]:
ST_C = ST_K - 273.15
model_inputs_df["ST_C"] = ST_C
print(np.nanmin(ST_C), np.nanmean(ST_C), np.nanmax(ST_C))

-14.42999999999995 29.030046929577487 86.11000000000001


In [16]:
NDVI = np.array(tower_data_df.NDVI)
model_inputs_df["NDVI"] = NDVI
print(np.nanmin(NDVI), np.nanmean(NDVI), np.nanmax(NDVI))

-0.02429185 0.4528892517239436 0.94546


In [17]:
Ta_C = np.array(tower_data_df.Ta)
model_inputs_df["Ta_C"] = Ta_C
print(np.nanmin(Ta_C), np.nanmean(Ta_C), np.nanmax(Ta_C))

-14.605048 22.321587967441317 39.710495


In [18]:
RH = np.array(tower_data_df.RH)
model_inputs_df["RH"] = RH
print(np.nanmin(RH), np.nanmean(RH), np.nanmax(RH))

0.27253073 0.42692000418779347 0.98366296


In [19]:
Rg = np.array(tower_data_df.Rg)
model_inputs_df["Rg"] = Rg
print(np.nanmin(Rg), np.nanmean(Rg), np.nanmax(Rg))

-23.763361 606.9121518676056 1042.9371


In [20]:
Rn = np.array(tower_data_df.Rn)
model_inputs_df["Rn"] = Rn
print(np.nanmin(Rn), np.nanmean(Rn), np.nanmax(Rn))

0.0 414.2791522507042 763.1359


In [21]:
albedo = np.array(tower_data_df.albedo)
model_inputs_df["albedo"] = albedo
print(np.nanmin(albedo), np.nanmean(albedo), np.nanmax(albedo))

0.015407953 0.10903688726384977 0.6173986


In [22]:
model_inputs_df

,ID,name,Topt,fAPARmax,geometry,ST_K,ST_C,NDVI,Ta_C,RH,Rg,Rn,albedo
0,US-NC3,NC_Clearcut#3,10.09,0.4659,POINT (-76.656 35.799),305.10,31.95,0.709729,32.658920,0.560215,545.51056,393.85710,0.215445
1,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865,POINT (-80.637 41.8222),304.34,31.19,0.605842,24.227982,0.458503,848.34390,643.47500,0.117238
2,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865,POINT (-80.637 41.8222),304.06,30.91,0.578494,26.178862,0.621679,838.81160,628.76870,0.117280
3,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865,POINT (-80.637 41.8222),301.80,28.65,0.658499,22.527096,0.623986,851.72480,626.95960,0.084629
4,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865,POINT (-80.637 41.8222),303.18,30.03,0.666747,23.280691,0.571613,702.55160,513.13380,0.120526
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,US-xAE,NEON Klemme Range Research Station (OAES),7.99,0.3387,POINT (-99.0588 35.4106),278.78,5.63,0.268441,3.815752,0.395740,286.84660,163.24739,0.092853
1061,US-xAE,NEON Klemme Range Research Station (OAES),7.99,0.3387,POINT (-99.0588 35.4106),293.28,20.13,0.225175,19.266186,0.397430,290.87400,166.19196,0.111844
1062,US-xAE,NEON Klemme Range Research Station (OAES),7.99,0.3387,POINT (-99.0588 35.4106),301.94,28.79,0.282996,32.110336,0.341227,352.19530,235.88159,0.106782
1063,US-xAE,NEON Klemme Range Research Station (OAES),7.99,0.3387,POINT (-99.0588 35.4106),290.72,17.57,0.284051,10.464681,0.349086,420.67880,247.87890,0.106775


In [23]:
model_inputs_df.keys()

Index(['ID', 'name', 'Topt', 'fAPARmax', 'geometry', 'ST_K', 'ST_C', 'NDVI',
       'Ta_C', 'RH', 'Rg', 'Rn', 'albedo'],
      dtype='object')

In [24]:
model_inputs_df.albedo

0       0.215445
1       0.117238
2       0.117280
3       0.084629
4       0.120526
          ...   
1060    0.092853
1061    0.111844
1062    0.106782
1063    0.106775
1064    0.113165
Name: albedo, Length: 1065, dtype: float64

In [25]:
results = process_PTJPL_table(model_inputs_df)
results

[2025-07-21 17:41:58 INFO] GEOS-5 FP working directory: /Users/gregoryhalverson/data/GEOS5FP
[2025-07-21 17:41:58 INFO] GEOS-5 FP download directory: ~/data/GEOS5FP


,ID,name,Topt,fAPARmax,geometry,ST_K,ST_C,NDVI,Ta_C,RH,Rg,Rn,albedo,Rn_soil,LE_soil,Rn_canopy,PET,LE_canopy,LE_interception,LE
0,US-NC3,NC_Clearcut#3,10.09,0.4659,POINT (-76.656 35.799),305.10,31.95,0.709729,32.658920,0.560215,545.51056,393.85710,0.215445,108.025913,20.624721,285.831187,348.579227,225.283244,28.623222,274.531187
1,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865,POINT (-80.637 41.8222),304.34,31.19,0.605842,24.227982,0.458503,848.34390,643.47500,0.117238,242.983670,46.430457,400.491330,518.484847,319.860529,16.324344,382.615330
2,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865,POINT (-80.637 41.8222),304.06,30.91,0.578494,26.178862,0.621679,838.81160,628.76870,0.117280,255.079320,100.956898,373.689380,518.782458,277.188284,52.841827,430.987009
3,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865,POINT (-80.637 41.8222),301.80,28.65,0.658499,22.527096,0.623986,851.72480,626.95960,0.084629,203.478663,84.178060,423.480937,505.753641,284.691299,57.763947,426.633306
4,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865,POINT (-80.637 41.8222),303.18,30.03,0.666747,23.280691,0.571613,702.55160,513.13380,0.120526,162.335322,52.805894,350.798478,413.852320,250.072976,34.076662,336.955533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,US-xAE,NEON Klemme Range Research Station (OAES),7.99,0.3387,POINT (-99.0588 35.4106),278.78,5.63,0.268441,3.815752,0.395740,286.84660,163.24739,0.092853,121.450797,39.624047,41.796593,92.388700,15.777073,0.595122,55.996242
1061,US-xAE,NEON Klemme Range Research Station (OAES),7.99,0.3387,POINT (-99.0588 35.4106),293.28,20.13,0.225175,19.266186,0.397430,290.87400,166.19196,0.111844,131.899739,30.509115,34.292221,128.608037,22.745904,0.729878,53.984898
1062,US-xAE,NEON Klemme Range Research Station (OAES),7.99,0.3387,POINT (-99.0588 35.4106),301.94,28.79,0.282996,32.110336,0.341227,352.19530,235.88159,0.106782,171.573784,6.705015,64.307806,207.238994,57.877762,0.881771,65.464547
1063,US-xAE,NEON Klemme Range Research Station (OAES),7.99,0.3387,POINT (-99.0588 35.4106),290.72,17.57,0.284051,10.464681,0.349086,420.67880,247.87890,0.106775,180.002902,48.505452,67.875998,161.062364,42.699731,0.711993,91.917177
